# Final Project of Introduction to Bioinformatics

## Find The Imposter - Deciphering Mysterious Sequences

#### TA: Javad Razi (j.razi@outlook.com)

## Project Description: The Genomic Detective - Delving into Avian DNA with Galaxy

### Overview

Welcome to an exploratory journey into the world of bioinformatics, where we will delve into the DNA of flying species. This project presents a unique opportunity to unravel a genomic mystery using Galaxy, a sophisticated yet user-friendly bioinformatics platform. Your mission is to assemble a genome from short-read sequences, revealing insights into a specific DNA sequence found in various avian species. Along the way, you'll learn to navigate the complexities of genome assembly and conduct detailed BLAST searches, piecing together a puzzle millions of years in the making. 

### Objectives and Workflow

1. **Introduction and Setup with Galaxy:**
   - Start by exploring the Galaxy platform, designed for bioinformatics analysis. You can find a comprehensive introduction and a step-by-step guide on how to use Galaxy, including how to set up your work environment and get data into Galaxy, at the [Galaxy Project Training Network](https://training.galaxyproject.org/). This resource provides a hands-on introduction to Genomics and Galaxy, covering basic aspects like creating a new history and using the Get Data toolbox.

2. **Genome Assembly:**
   - For learning about genome assembly methods, the [Galaxy Project Training Network](https://training.galaxyproject.org/) offers a variety of resources and guides. This site provides access to a wide range of learning materials, helping users to understand the intricacies of genome assembly within the Galaxy platform.

3. **Performing BLAST Searches:**
   - To understand how to perform BLAST searches using Galaxy, the NCBI BLAST User Guide remains a crucial resource. You can access it at [NCBI's BLAST User Guide](https://www.ncbi.nlm.nih.gov/books/NBK279690/). This guide offers detailed instructions and insights into using BLAST for sequence comparison and analysis.

4. **Comparative Genomics and Analysis:**
   - Compare your findings against existing genomic data. This comparative analysis will help you shed light on the unique aspects of your assembled sequence and its significance in avian genetics.

### Specific Deliverables

- **Complete Code:** Submit all the code you used for assembling the genome, performing BLAST searches, and further analysis. Ensure your code is well-commented and organized for clarity.
- **Assembled Genome Fasta File:** Provide the fasta file of the assembled genome. This should be the direct output of your assembly process.
- **BLAST Results CSV File:** Include a CSV file with the results from your BLAST searches. This file should contain detailed information about any genomic matches found.
- **Detailed Interpretation:** At the end of your notebook, include a thorough interpretation of your findings. Discuss the significance of the sequence within the avian genome, any similarities or differences with sequences in other species, and the potential implications of these results. Your interpretation should be grounded in the data analysis conducted.

In [2]:
import sys
import subprocess
import pkg_resources

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

REQUIRED_PACKAGES = [
    'bioblend',
    'biopython',
    'pandas'
]

for package in REQUIRED_PACKAGES:
    try:
        dist = pkg_resources.get_distribution(package)
        print('{} ({}) is installed'.format(dist.key, dist.version))
    except pkg_resources.DistributionNotFound:
        print('{} is NOT installed'.format(package))
        install(package)
        print('{} was successfully installed.'.format(package))

bioblend is NOT installed


CalledProcessError: Command '['c:\\Users\\morte\\AppData\\Local\\Programs\\Python\\Python312\\python.exe', '-m', 'pip', 'install', 'bioblend']' returned non-zero exit status 1.

## Part 1: Assembling Using Galaxy

#### Option 1: Python Notebook

Finish this section of notebook to assemble a genome from a fasta file with short-read sequences.

#### Option 2: Galaxy Web Interface

Alternatively, you can use the Galaxy web interface at usegalaxy.org to complete the assembly. This approach allows you to experience the ease and efficiency of Galaxy's web-based tools.


In [4]:
!python import sys !{sys.executable} -m pip install bioblend

python: can't open file 'd:\\university\\Master\\Term 1\\Introduction to bioinformatics\\Introduction to bioinformatics\\Project\\bioinformatics-project\\find-the-imposter\\import': [Errno 2] No such file or directory


In [41]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv(dotenv_path='../../.env')

# You can create your API key by registering at usegalaxy website, and from user settings section.
# It is recommended that you store this key as an environment variable, and not expose it!
api_key = "a52a7712bfc212b60514205bfc79191e"

In [42]:
import bioblend.galaxy

# Initialize Galaxy instance
gi = bioblend.galaxy.GalaxyInstance(url='https://usegalaxy.org', key=api_key)

In [43]:
# Upload the fasta file to Galaxy
fasta_path = '/content/short_reads.fasta'
fasta_hist = gi.histories.create_history(name="FindTheImposterTask_Fasta")
fasta_dict = gi.tools.upload_file(fasta_path, fasta_hist['id'], type='fasta')

fasta_dict

{'outputs': [{'id': 'f9cad7b01a472135263933625a4631d3',
   'hda_ldda': 'hda',
   'uuid': '97b2d938-8ac2-42a3-ad43-0e8a328e355e',
   'hid': 1,
   'file_ext': 'auto',
   'peek': None,
   'model_class': 'HistoryDatasetAssociation',
   'name': 'short_reads.fasta',
   'deleted': False,
   'purged': False,
   'visible': True,
   'state': 'queued',
   'history_content_type': 'dataset',
   'file_size': 0,
   'create_time': '2024-01-19T01:54:40.412132',
   'update_time': '2024-01-19T01:54:40.432578',
   'data_type': 'galaxy.datatypes.data.Data',
   'genome_build': '?',
   'validated_state': 'unknown',
   'validated_state_message': None,
   'misc_info': None,
   'misc_blurb': None,
   'tags': [],
   'history_id': '97bfdd7e3a68efa7',
   'metadata_dbkey': '?',
   'output_name': 'output0'}],
 'output_collections': [],
 'jobs': [{'model_class': 'Job',
   'id': 'bbd44e69cb8906b5b8a5fbbeb6bc3bd6',
   'state': 'new',
   'exit_code': None,
   'update_time': '2024-01-19T01:54:40.467496',
   'create_time'

In [45]:
# Identify and Prepare the Assembly Tool

# Retrieve SPAdes assembly tool from Galaxy's tool repository.
assembly_tool_name = 'SPAdes'
assembly_tool = gi.tools.get_tools(name=assembly_tool_name)[0]

# Prepare inputs for the SPAdes assembly tool
assembly_params = {
    # Set the mode for the assembly. SPAdes has different modes like 'careful', 'normal', etc.
    # Refer to SPAdes documentation to understand what modes are available and what each mode does.
    'mode_sel': '--careful',
    
    # Specify the operation mode. SPAdes allows different operations like error correction, only assembly, etc.
    # Select the one that aligns with your project requirements.
    'operation_mode': '--only-assembler',
}

# Run the SPAdes Assembly Tool
try:
    assembly_dict = gi.tools.run_tool(fasta_hist['id'], assembly_tool['id'], assembly_params)
except ConnectionError as e:
    print(f"Failed to run the assembly tool: {e}")
    raise

# Wait for the assembly job to complete
assembly_dict = gi.jobs.wait_for_job(job_id=assembly_dict['jobs'][0]['id'], maxwait=120, interval=5, check=True)

In [ ]:
assembly_dict

In [73]:
# Download the assembled genome from Galaxy. You can use the `download_dataset` method. A FASTA file, containing assembly
# of the whole sequence is what we expect here. 

In [ ]:
gi.datasets.download_dataset('f9cad7b01a4721351fcb1f6e1d6576f1', file_path='.')

### Part 2: Using BLAST to Query The Assembled Sequence

In this part of the notebook, you will utilize the NCBI BLAST API to analyze the genome sequence you've assembled. This involves integrating the API into your notebook, submitting your sequence for BLAST querying, and then meticulously examining the results. Your focus will be on identifying similarities or unique traits in the sequence compared to others in the NCBI database, particularly exploring its relationship with known sequences in various species. This step is crucial for understanding the evolutionary and biological significance of your assembled genome.

**Note**: Unlike the previous section, for this one, you must deliver the full code in the notebook. Doing this part using website will not be graded. 

In [74]:
# Import necessary libraries
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
from collections import defaultdict



In [ ]:
# Load the assembled genome
with open('/content/Galaxy11-[SPAdes_on_data_1__Contigs].fasta', 'r') as file:
    assembled_genome = file.read()

assembled_genome

In [89]:
from Bio.Blast import NCBIWWW

# Perform the BLAST query, filtering for eukaryotes
eukaryote_tax_id = "2759"

### TODO ###
result_handle = NCBIWWW.qblast(program='blastn',
                               database='nr',
                               sequence=assembled_genome,
                               entrez_query='', # Only filter Eucaryotes. Hint: You can do this by giving their taxonomy id.
                               hitlist_size=100,
                               word_size=16)
### TODO ###


In [90]:
from Bio.Blast import NCBIXML

blast_records = NCBIXML.parse(result_handle)

blast_records

<generator object parse at 0x0000020D4D9B1540>

In [ ]:
import pandas as pd
from Bio import Entrez

# Set your email here for Entrez
Entrez.email = "your.email@example.com"

def fetch_taxonomy_info(accession):
    """
    Fetch taxonomy information using Entrez for a given accession number.
    """
    handle = Entrez.efetch(db="nucleotide", id=accession, retmode="xml")
    records = Entrez.read(handle)

    ### TODO ###

    taxonomy = records[0]["GBSeq_taxonomy"]
    species = records[0]["GBSeq_organism"]

    ### TODO ###

    return taxonomy, species


def parse_blast_results():
    """
    Parse BLAST results and extract relevant information including taxonomy.
    """
    blast_results = []

    for record in blast_records:
        for alignment in record.alignments:
            accession = alignment.accession
            taxonomy, species = fetch_taxonomy_info(accession)
            for hsp in alignment.hsps:
                # These fields are required in your submission
                blast_results.append({
                    'query_id': record.query_id,
                    'alignment_title': alignment.title,
                    'e_value': hsp.expect,
                    'identity': hsp.identities,
                    'accession': accession,
                    'taxonomy': taxonomy,
                    'species': species
                })
    return blast_results


blast_results = parse_blast_results()
df = pd.DataFrame(blast_results)
df.to_csv('./outputs/blast_results_with_taxonomy.csv', index=False)

df.head()

## Analysis of The Results

### Drawing Your Own Conclusions

Now that you have completed the BLAST search, a fascinating part of your journey begins – interpreting the data. This stage is where your critical thinking and creativity come into play. From now on, the rest of the notebook will be about whatever you want it to be. Any path that leads to meaningful insights about the data and provides a solid conclusion for the task is acceptable. Let's explore some possible directions:

1. **Species-Specific Patterns:** Examine if the sequence is found exclusively or predominantly in certain species. What could this suggest about its evolution and adaptation? While the focus is not on finding a 'correct' answer, pondering this aspect can lead to interesting hypotheses about species-specific interactions.

2. **Functional Insights:** Reflect on the potential roles this sequence might play within the genomes where it's found. Could it be integral to certain biological functions, or a legacy of ancient genomic events?

3. **Comparative Genomics:** Compare your findings with sequences in other species. Notice any striking similarities or differences? These comparisons could shed light on the sequence's evolutionary journey.

4. **Ecological and Environmental Context:** Consider the ecological and environmental factors that might influence the distribution and evolution of this sequence. How might habitat or lifestyle of the species play a role in its presence or absence?

### Additional Tips and Encouragement

This project is more about the learning journey and less about achieving perfect results. Here are some additional pointers:

1. **Deep Dives:** Encourage yourself to explore the data thoroughly. Use various bioinformatics tools to gain a holistic understanding.

2. **Creative Visualization:** Craft visual representations of your analysis. Effective use of charts or infographics can provide insightful perspectives.

3. **Open-Ended Exploration:** Feel free to extend your analysis in directions you find intriguing. This could include phylogenetic studies or exploring the ecological aspects of the sequence.

Remember, this project is designed to be a learning experience. We don't expect you to uncover all the answers but rather to engage thoughtfully with the data and enjoy the process of discovery.

In [ ]:
### TODO ###